# Data Exploration

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv("data/electricity_demand_data.csv")
df.head()

To see what types of data 

In [ ]:
df.info()

No missing data

In [ ]:
df.isna().sum()

Peak demand value

In [ ]:
max(df.demand)

In [ ]:
round(df.describe(),2)

Below we can see that device_id, Postcode and property_category are constant throughout the dataset, so unless we can gather information helping our analysis from these single results, they are not useful for our modelling predictions


In [ ]:
df.nunique()

In [ ]:
print(f"Date range of elecriticty demand available from - {df.loc[:,'DateTime'][0]} to {df.loc[:,'DateTime'][len(df) -1]}")

In [ ]:
df.demand.plot()

Here we can see some outliers in the demand at certain points

In [ ]:
df[df.demand > 4]

With a quick google search of the news on that date we can see there are strikes in Germany strikes? 06:00 April 17-06:00 April 21
 The third largest strike took place on 20 April 2023:

However these due to the magntitude of the demand at these peaks, I imagine it is more likely collection issues and regardless unhelpful for our modelling
 


In [11]:
df['DateTime'] = pd.to_datetime(df['DateTime'])

In [ ]:
df['minute'] = df['DateTime'].dt.minute
df['hour'] = df['DateTime'].dt.hour
df['year'] = df['DateTime'].dt.year
df['month'] = df['DateTime'].dt.month
df['date'] = df['DateTime'].dt.date
df['weekday'] = df['DateTime'].dt.day_name()



plt.figure(figsize=(10, 6))
sns.countplot(y="minute", data=df)
plt.title("Distribution of demand recordings by minute")
plt.xlabel("Quantities of recordings")
plt.ylabel("Minute in hour")
plt.show()


We can see that primarily recordings are made at 0,15,30,45 past the hour

In [ ]:
minute_means = df.groupby('minute')['demand'].mean()

plt.figure(figsize=(10, 6))
plt.plot(minute_means.index, minute_means.values, marker='o', label='Mean Demand per Minute')
plt.title('Mean Demand for each minute in hour recorded')
plt.xlabel('Minute in hour')
plt.ylabel('Mean Demand')
plt.grid(True)
plt.xticks(minute_means.index)
plt.tight_layout()
plt.show()

And at these readings at 0,15,30,45 past the hour, demand is recorded to be higher, perhaps the readings not at those times are at a higher frequency, and so record lower demand

In [ ]:
df[~df['minute'].isin([0, 15, 30,45])].year.value_counts()

readings that aren't at 0,15,30,45 minutes past the hour seem to only occur from 2023 onwards

In [ ]:
df_2023 = df[(df.year == 2023) & (df.month == 10)]
minute_means = df_2023.groupby('minute')['demand'].mean()

plt.figure(figsize=(10, 6))
plt.plot(minute_means.index, minute_means.values, marker='o', label='Mean Demand per Minute')
plt.title('Mean Demand for Each Minute')
plt.xlabel('Minute')
plt.ylabel('Mean Demand')
plt.grid(True)
plt.xticks(minute_means.index)
plt.show()

And in this plot we can see the discrepency between 0,15,30,45 minutes past the hour and other time readings is smaller, the mean demand at those times are lower than in previous years when off those times weren't recorded

In [ ]:
df['date'] = pd.to_datetime(df['date'])
readings_per_day = df.groupby('date').size().reset_index(name='readings')
readings_per_day['day_of_week'] = readings_per_day['date'].dt.dayofweek
avg_readings_per_day = readings_per_day.groupby('day_of_week')['readings'].mean().reset_index()

plt.figure(figsize=(8,6))
plt.bar(avg_readings_per_day['day_of_week'], avg_readings_per_day['readings'])
plt.ylabel('Average Number of Readings')
plt.title('Average Number of Readings Per Day of the Week')
plt.xticks(ticks=range(7), labels=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.show()

readings per day remain pretty constant

In [ ]:
df['date'] = pd.to_datetime(df['date'])
readings_per_day = df.groupby('date').size().reset_index(name='readings')
readings_per_day['month'] = readings_per_day['date'].dt.month
avg_readings_per_month = readings_per_day.groupby('month')['readings'].mean().reset_index()

plt.figure(figsize=(8,6))
plt.bar(avg_readings_per_month['month'], avg_readings_per_month['readings'])
plt.xlabel('Month')
plt.ylabel('Average Number of Readings')
plt.title('Average Number of Readings Per Month')
plt.xticks(ticks=range(1, 13), labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.show()

But readings per month dips a little in the summer months

In [ ]:

df_cut = df[df.date < '2023-09-01']
df_cut['date'] = pd.to_datetime(df_cut['date'])
readings_per_day = df_cut.groupby('date').size().reset_index(name='readings')
readings_per_day['month'] = readings_per_day['date'].dt.month
avg_readings_per_month = readings_per_day.groupby('month')['readings'].mean().reset_index()

plt.figure(figsize=(8,6))
plt.bar(avg_readings_per_month['month'], avg_readings_per_month['readings'])
plt.xlabel('Month')
plt.ylabel('Average Number of Readings')
plt.title('Average Number of Readings Per Month pre september 2023')
plt.xticks(ticks=range(1, 13), labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.show()

Here we check to see if the shape of readings per month was changed because only in the last few months of the data did we see a higher frequency of readings per hour

In [ ]:

df['date'] = pd.to_datetime(df['date'])
readings_per_day = df.groupby('date').size().reset_index(name='readings')
readings_per_day['year_month'] = readings_per_day['date'].dt.to_period('M')  # 'YYYY-MM' format
avg_readings_per_month = readings_per_day.groupby('year_month')['readings'].mean().reset_index()
avg_readings_per_month['year_month'] = avg_readings_per_month['year_month'].dt.to_timestamp()

plt.figure(figsize=(12,6))
plt.plot(avg_readings_per_month['year_month'], avg_readings_per_month['readings'], marker='o')
plt.xlabel('Year and Month')
plt.ylabel('Average Number of Readings per Day')
plt.title('Average Number of Readings Per Day for Each Month Over 3 Years')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
df.nunique()

Below we have removed our few anomalies in the demand column to make the rest of the data more digestible 

In [21]:
df = df[df['demand'] < 4]

In [ ]:
fig, ax = plt.subplots(figsize = (10,8))
sns.boxplot(data = df, x = 'hour', y = 'demand' )
ax.set_title('demand by hour')
plt.show()

The mean reading by hour remains much lower than a lot of the readings, with a higher deviation around midday, suggesting lots of low usages and more peaks around midday

In [ ]:
fig, ax = plt.subplots(figsize = (10,8))
sns.boxplot(data = df, x = 'month', y = 'demand' )
ax.set_title('demand by month')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (10,8))
sns.boxplot(data = df, x = 'weekday', y = 'demand' )
ax.set_title('demand by day of week')
plt.show()